In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/CS2/train.zip -d train

Archive:  /content/drive/MyDrive/CS2/train.zip
  inflating: train/00204008d.flac    
  inflating: train/003b04435.flac    
  inflating: train/003bec244.flac    
  inflating: train/005f1f9a5.flac    
  inflating: train/006ab765f.flac    
  inflating: train/0072f0839.flac    
  inflating: train/0079ff47b.flac    
  inflating: train/007f87ba2.flac    
  inflating: train/00834f88e.flac    
  inflating: train/008c9c381.flac    
  inflating: train/0099c367b.flac    
  inflating: train/009aa1c8f.flac    
  inflating: train/009b760e6.flac    
  inflating: train/009fbc7b4.flac    
  inflating: train/00ad36516.flac    
  inflating: train/00b404881.flac    
  inflating: train/00d442df7.flac    
  inflating: train/00e3525a2.flac    
  inflating: train/00f3b3c1e.flac    
  inflating: train/011f25080.flac    
  inflating: train/013716dbf.flac    
  inflating: train/015113cad.flac    
  inflating: train/0151b7d20.flac    
  inflating: train/015aa6c7c.flac    
  inflating: train/016240b9a.flac    
  i

In [ ]:
! unzip /content/drive/MyDrive/CS2/test.zip -d test

Archive:  /content/drive/MyDrive/CS2/test.zip
  inflating: test/000316da7.flac     
  inflating: test/003bc2cb2.flac     
  inflating: test/0061c037e.flac     
  inflating: test/010eb14d3.flac     
  inflating: test/011318064.flac     
  inflating: test/01307e693.flac     
  inflating: test/013bd8b85.flac     
  inflating: test/01812f522.flac     
  inflating: test/019db5220.flac     
  inflating: test/01bd97dd6.flac     
  inflating: test/01d36def5.flac     
  inflating: test/01fefebcb.flac     
  inflating: test/0208fec47.flac     
  inflating: test/0232841d4.flac     
  inflating: test/02c30604d.flac     
  inflating: test/02dbcc383.flac     
  inflating: test/0303dee3c.flac     
  inflating: test/034f46152.flac     
  inflating: test/037460d90.flac     
  inflating: test/039395460.flac     
  inflating: test/039892ae3.flac     
  inflating: test/03a06a709.flac     
  inflating: test/03a9a5b2a.flac     
  inflating: test/03d0c7428.flac     
  inflating: test/03f5a4505.flac     
  in

In [ ]:
! pip install -q tensorflow-io

     |████████████████████████████████| 25.3MB 137kB/s 


In [ ]:
! pip install tensorflow-addons

In [ ]:
! pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-o3f668_7
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-o3f668_7
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20030 sha256=e4e1f91b46f2407c400714e41509c2c18f337d21fabbbbb7433fc59f283d37a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-j44bku4q/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


## 1. Importing necessary libraries


In [ ]:
import tensorflow as tf
tf.version.VERSION

'2.4.0'

In [ ]:
# Importing the necessary libraries
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_addons as tfa
import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
from classification_models.keras import Classifiers
from sklearn.metrics import label_ranking_average_precision_score

from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# Global variables 
HEIGHT = 384
WIDTH = 768
SR = 48000
CLASS_N = 24
MELS = 384
FMIN = 40
FMAX = 24000
BATCHSIZE = 8
STEPS_PER_EPOCH = 140 #(train_pts/batch_size)

## 2. Loading dataset

In [ ]:
# Loading the dataset 
train_tp = pd.read_csv('/content/drive/MyDrive/CS2/train_tp.csv')
train_tp

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max
0,003bec244,14,1,44.5440,2531.250,45.1307,5531.25
1,006ab765f,23,1,39.9615,7235.160,46.0452,11283.40
2,007f87ba2,12,1,39.1360,562.500,42.2720,3281.25
3,0099c367b,17,4,51.4206,1464.260,55.1996,4565.04
4,009b760e6,10,1,50.0854,947.461,52.5293,10852.70
...,...,...,...,...,...,...,...
1211,fe8d9ac40,13,1,53.4720,93.750,54.0960,843.75
1212,fea6b438a,4,1,43.5787,2531.250,45.7653,4031.25
1213,ff2eb9ce5,0,1,15.2267,5906.250,16.0213,8250.00
1214,ffb8d8391,5,1,14.3467,4781.250,16.6987,10406.20


In [ ]:
# Loading the test data files
test = os.listdir('test/')  

In [ ]:
# Splitting the train true positive data into train and validation
train, validation = train_test_split(train_tp, test_size = 0.1, stratify = train_tp['species_id'])

## 3. Creating data pipelines

In [ ]:
 def create_mel_spectrogram(file):
    """ This function reads the file and computes the mel spectrogram and returns the spectrogram"""
    audio = tfio.IOTensor.graph(tf.int16).from_audio(file)
    audio_slice = audio[100:]

    # remove last dimension
    audio_tensor = tf.squeeze(audio_slice, axis=[-1])
    audio_tensor = tf.cast(audio_tensor, tf.float32)

    # Convert to spectrogram
    spectrogram = tfio.experimental.audio.spectrogram(audio_tensor, nfft=2048, window=2048, stride=512)

    # Convert to mel-spectrogram
    mel_spectrogram = tfio.experimental.audio.melscale(spectrogram, rate=SR, mels=MELS, fmin=FMIN, fmax=FMAX)

    # Convert to db scale mel-spectrogram
    mel_spectrogram = tfio.experimental.audio.dbscale(mel_spectrogram, top_db=80)

    # Expanding the dimensions of spectrograms by 1 
    image = tf.expand_dims(mel_spectrogram, axis= -1)
    # Resizing the spectrogram
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    # Converting the spectrogram to rgb
    image = tf.image.grayscale_to_rgb(image)

    # Returning mel_spectrogram
    return image

In [ ]:
def augmentation(spectrogram, label):
    """In this function we resize the image and perform data augumentations for train data"""

    # Getting a random value and based on the random value computing the augmentation
    a = np.random.uniform()
    if a<0.25:
        # Flipping the spectrogram left to right
        spectrogram = tf.image.flip_left_right(spectrogram)
    elif a<0.5:
        # Flipping the spectrogram up to down
        spectrogram = tf.image.flip_up_down(spectrogram)
    elif a<0.75:
        # Adding a random contrast to spectrogram
        spectrogram = tf.image.random_contrast(spectrogram, 0.2,0.5)
    else:
        # Adding a random brightness to spectrogram
        spectrogram = tf.image.random_brightness(spectrogram, 0.2)
    # else :
    #     # Adding gaussian noise to spectrogram
    #     gn = tf.keras.layers.GaussianNoise(0.3, dtype = tf.float64)
    #     spectrogram = gn(spectrogram, training = True)

    # Reducing the dimensions of the spectrogram
    # spectrogram = tf.squeeze(spectrogram)

    # Returning the spectrogram image
    return spectrogram, label

In [ ]:
def preprocess_train(file):
    """Preprocessing train files"""
    # Calling the preprocess_img function with data_type as train
    image = create_mel_spectrogram('train/'+file['recording_id']+'.flac')
    # Computing the one hot encoded values of species_id
    label = tf.one_hot(file['species_id'], CLASS_N)
    # Returning the spectrogram and one hot encoded species_id
    return image, label


def preprocess_val(file):
    """Preprocessing validation files"""
    # Calling the preprocess_img function with data_type as validation
    image = create_mel_spectrogram('train/'+file['recording_id']+'.flac')

    # Computing the one hot encoded values of species_id
    label = tf.one_hot(file['species_id'], CLASS_N)
    # Returning the spectrogram and one hot encoded species_id
    return image, label

    
def preprocess_test(file):
    """Preprocessing test files"""
    # Calling the preprocess_img function with data_type as test
    image = create_mel_spectrogram('test/'+file)

    file_name = tf.strings.split(file,'.')[0]
    # Returning only the spectrogram image for test dataset 
    return image, file_name

In [ ]:
# Creating a dataset from train data and mapping the preprocess train function 
files_ds = tf.data.Dataset.from_tensor_slices(dict(train))
train_dataset = files_ds.map(preprocess_train).cache().map(augmentation).shuffle(3).batch(BATCHSIZE).prefetch(-1).repeat()

# Creating a dataset from train data and mapping the preprocess train function 
files_ds = tf.data.Dataset.from_tensor_slices(dict(validation))
val_dataset = files_ds.map(preprocess_val).cache().shuffle(3).batch(BATCHSIZE).prefetch(-1)

## 4. Training the model

In [ ]:
def lrap(y_true, y_pred):
    return tf.py_function(label_ranking_average_precision_score, (y_true, y_pred), tf.double)

In [ ]:
backbone = tf.keras.applications.DenseNet121(include_top = False,input_shape = (HEIGHT,WIDTH,3), weights="imagenet")

for layer in backbone.layers[:0]:
    layer.trainable = False

model = tf.keras.Sequential([
            backbone,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.he_normal()),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(CLASS_N,bias_initializer=tf.keras.initializers.Constant(-2.))])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 12, 24, 1024)      7037504   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               131200    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                3096      
Total params: 7,172,312
Trainable params: 7,088,408
Non-trainable params: 83,904
_________________________________________

In [ ]:
model.compile(optimizer=tfa.optimizers.RectifiedAdam(learning_rate=2e-3,total_steps=30*140,warmup_proportion=0.3,min_lr=1e-6), loss=tfa.losses.SigmoidFocalCrossEntropy(from_logits = True), metrics=[lrap])

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='/content/drive/My Drive/CS2/Logs/Densenet121_not_overfitting',histogram_freq=1, write_graph=True,write_grads=True),
    tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/CS2/Saved Models/Densenet121_not_overfitting/{epoch:02d}-{lrap:.4f}-{val_lrap:.4f}.h5', save_weights_only=True)]

In [ ]:
model.fit(train_dataset,steps_per_epoch=STEPS_PER_EPOCH, validation_data=val_dataset,epochs = 30,callbacks = callbacks)

Epoch 1/30
140/140 [==============================] - 1758s 12s/step - loss: 0.7873 - lrap: 0.1536 - val_loss: 0.5142 - val_lrap: 0.1772
Epoch 2/30
140/140 [==============================] - 87s 621ms/step - loss: 0.6061 - lrap: 0.2042 - val_loss: 0.9257 - val_lrap: 0.1669
Epoch 3/30
140/140 [==============================] - 89s 637ms/step - loss: 0.5044 - lrap: 0.2575 - val_loss: 3.1844 - val_lrap: 0.1654
Epoch 4/30
140/140 [==============================] - 90s 643ms/step - loss: 0.4413 - lrap: 0.2931 - val_loss: 0.6520 - val_lrap: 0.2192
Epoch 5/30
140/140 [==============================] - 90s 642ms/step - loss: 0.3970 - lrap: 0.3405 - val_loss: 0.4463 - val_lrap: 0.2827
Epoch 6/30
140/140 [==============================] - 91s 650ms/step - loss: 0.3837 - lrap: 0.3506 - val_loss: 0.8846 - val_lrap: 0.1826
Epoch 7/30
140/140 [==============================] - 91s 649ms/step - loss: 0.3571 - lrap: 0.3853 - val_loss: 1.5927 - val_lrap: 0.1670
Epoch 8/30
140/140 [=====================